In [1]:
import ebooklib
from ebooklib import epub

def epub2thtml(epub_path):
    book = epub.read_epub(epub_path)
    chapters = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters.append(item.get_content())
    return chapters

from bs4 import BeautifulSoup
blacklist = ['[document]', 'noscript', 'header', 'html', 'meta', 'head', 'input', 'script', ]
# there may be more elements you don't want, such as "style", etc.

def chap2text(chap):
    output = ''
    soup = BeautifulSoup(chap, 'html.parser')
    text = soup.find_all(text=True)
    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    return output

def thtml2ttext(thtml):
    Output = []
    for html in thtml:
        text =  chap2text(html)
        Output.append(text)
    return Output

def epub2text(epub_path):
    chapters = epub2thtml(epub_path)
    ttext = thtml2ttext(chapters)
    return ttext

In [2]:
harrypotter=epub2text('Harry Potter.epub')
davinci=epub2text('Da Vinci Code.epub')
clouds=epub2text('Head in the Clouds.epub')
cosmos=epub2text("Cosmos.epub")

In [3]:
#harrypotter

In [4]:
#davinci

In [5]:
#clouds

In [6]:
#cosmos

In [7]:
import pandas as pd
import re

harrypotter_text = ''
for o in harrypotter:
    harrypotter_text = harrypotter_text + o
    
harrypotter_lines = harrypotter_text.splitlines()
harrypotter_df = pd.DataFrame({
    "content": harrypotter_lines,
    "line": list(range(len(harrypotter_lines)))
})
harrypotter_df

,content,line
0,,0
1,,1
2,,2
3,,3
4,HP 4 - Harry Potter and The Goblet of Fire,4
...,...,...
6764,"Har­ry winked at them, turned to Un­cle...",6764
6765,"As Ha­grid had said, what would come, w...",6765
6766,,6766
6767,,6767


In [8]:
davinci_text = ''
for o in davinci:
    davinci_text = davinci_text + o
    
davinci_lines = davinci_text.splitlines()
davinci_df = pd.DataFrame({
    "content": davinci_lines,
    "line": list(range(len(davinci_lines)))
})
davinci_df

,content,line
0,,0
1,The Da Vinci Code,1
2,Dan Brown,2
3,,3
4,FOR BLYTHE... AGAIN. MORE THAN EVER.,4
...,...,...
5887,,5887
5888,,5888
5889,,5889
5890,,5890


In [9]:
clouds_text = ''
for o in clouds:
    clouds_text = clouds_text + o
    
clouds_lines = clouds_text.splitlines()
clouds_df = pd.DataFrame({
    "content": clouds_lines,
    "line": list(range(len(clouds_lines)))
})
clouds_df

,content,line
0,,0
1,Head,1
2,IN THE,2
3,Clouds,3
4,,4
...,...,...
266,,266
267,,267
268,,268
269,,269


In [10]:
cosmos_text = ''
for o in cosmos:
    cosmos_text = cosmos_text + o
    
cosmos_lines = cosmos_text.splitlines()
cosmos_df = pd.DataFrame({
    "content": cosmos_lines,
    "line": list(range(len(cosmos_lines)))
})
cosmos_df

,content,line
0,ACKNOWLEDGMENTS,0
1,"Besides those thanked in the introduction, I ...",1
2,,2
3,By Carl Sagan Published by The Random House P...,3
4,,4
...,...,...
1703,,1703
1704,,1704
1705,,1705
1706,,1706


In [11]:
def line_is_chapter_harrypotter(dataframe):
    chapter_list = []
    curr_chapter = 0
    for index, row in dataframe.iterrows():
        if re.search("^CHAPTER ONE|TWO|THREE|FOUR|FIVE|SIX|SEVEN|EIGHT|NINE|TEN|ELEVEN|TWELVE|.*TEEN|TWENTY\-?[A-Z]*|THIRTY\-?[A-Z]*", row['content']):
            curr_chapter += 1
        chapter_list.append(curr_chapter)
    return chapter_list

curr_chapter = 0
harrypotter_df = harrypotter_df.assign(chapter = line_is_chapter_harrypotter(harrypotter_df))
harrypotter_df

,content,line,chapter
0,,0,0
1,,1,0
2,,2,0
3,,3,0
4,HP 4 - Harry Potter and The Goblet of Fire,4,0
...,...,...,...
6764,"Har­ry winked at them, turned to Un­cle...",6764,37
6765,"As Ha­grid had said, what would come, w...",6765,37
6766,,6766,37
6767,,6767,37


In [12]:
def line_is_chapter_davinci(dataframe):
    chapter_list = []
    curr_chapter = 0
    for index, row in dataframe.iterrows():
        if re.search("Prologue|Epilogue|CHAPTER [\d]+|HOUSE", row['content']):
            curr_chapter += 1
        chapter_list.append(curr_chapter)
    return chapter_list

curr_chapter = 0
davinci_df = davinci_df.assign(chapter = line_is_chapter_davinci(davinci_df))
davinci_df

,content,line,chapter
0,,0,0
1,The Da Vinci Code,1,0
2,Dan Brown,2,0
3,,3,0
4,FOR BLYTHE... AGAIN. MORE THAN EVER.,4,0
...,...,...,...
5887,,5887,109
5888,,5888,109
5889,,5889,109
5890,,5890,109


In [13]:
def line_is_chapter_clouds(dataframe):
    chapter_list = []
    curr_chapter = 0
    for index, row in dataframe.iterrows():
        if re.search("^Prologue|Chapter [\d]+|About the Author", row['content']):
            curr_chapter += 1
        chapter_list.append(curr_chapter)
    return chapter_list

curr_chapter = 0
clouds_df = clouds_df.assign(chapter = line_is_chapter_clouds(clouds_df))
clouds_df = clouds_df[0:257]
clouds_df

,content,line,chapter
0,,0,0
1,Head,1,0
2,IN THE,2,0
3,Clouds,3,0
4,,4,0
...,...,...,...
252,,252,44
253,Chapter 44 O NE MONTH LATER China cups clink...,253,45
254,,254,45
255,,255,45


In [14]:
def line_is_chapter_cosmos(dataframe):
    chapter_list = []
    curr_chapter = 0
    for index, row in dataframe.iterrows():
        if re.search("CHAPTER [IVX]+|Chapter V", row['content']):
            curr_chapter += 1
        chapter_list.append(curr_chapter)
    return chapter_list

curr_chapter = 0
cosmos_df = cosmos_df.assign(chapter = line_is_chapter_cosmos(cosmos_df))
cosmos_df

,content,line,chapter
0,ACKNOWLEDGMENTS,0,0
1,"Besides those thanked in the introduction, I ...",1,0
2,,2,0
3,By Carl Sagan Published by The Random House P...,3,0
4,,4,0
...,...,...,...
1703,,1703,13
1704,,1704,13
1705,,1705,13
1706,,1706,13


In [16]:
from tidytext import unnest_tokens
import nltk
from nltk.corpus import stopwords
from siuba import *

harrypotter_df = (unnest_tokens(harrypotter_df, "word", "content"))
harrypotter_df = harrypotter_df.reset_index()
harrypotter_df = harrypotter_df.drop(columns=['index'])


harrypotter_df

,line,chapter,word
0,0,0,NaN
1,1,0,NaN
2,2,0,NaN
3,3,0,NaN
4,4,0,hp
...,...,...,...
202560,6765,37,it
202561,6765,37,did
202562,6766,37,NaN
202563,6767,37,NaN


In [23]:
import nltk
from nltk.corpus import stopwords
from siuba import *
punctuation_signs = ['\'', '"', ',', '.', '!', '?', '/', '[', ']', '(',')', '“', '”', '’', '‘']

harrypotter_words = count(harrypotter_df, _.chapter, _.word, sort=True)
harrypotter_words

,chapter,word,n
0,31,har­ry,158
1,26,har­ry,154
2,28,har­ry,148
3,23,har­ry,141
4,20,har­ry,140
...,...,...,...
47670,20,busy,1
47671,20,bush,1
47672,20,burst,1
47673,20,burned,1


In [24]:

harrypotter_df = filter(harrypotter_df, ~_.word.isin(stopwords.words('english')))
harrypotter_df = filter(harrypotter_df,~_.word.isin(punctuation_signs))
harrypotter_df

,line,chapter,word
0,0,0,NaN
1,1,0,NaN
2,2,0,NaN
3,3,0,NaN
4,4,0,hp
...,...,...,...
202554,6765,37,would
202557,6765,37,meet
202562,6766,37,NaN
202563,6767,37,NaN


In [26]:
harrypotter_df = harrypotter_df[harrypotter_df.word.notnull()]
harrypotter_df = harrypotter_df.reset_index()
harrypotter_df = harrypotter_df.drop(columns=['index'])
harrypotter_df

,line,chapter,word
0,4,0,hp
1,4,0,4
2,4,0,harry
3,4,0,potter
4,4,0,goblet
...,...,...,...
106828,6765,37,come
106829,6765,37,would
106830,6765,37,come…and
106831,6765,37,would


In [27]:
from tidytext import unnest_tokens
import nltk
from nltk.corpus import stopwords
from siuba import *

davinci_df = (unnest_tokens(davinci_df, "word", "content"))
davinci_df = davinci_df.reset_index()
davinci_df = davinci_df.drop(columns=['index'])

davinci_words = count(davinci_df, _.chapter, _.word, sort=True)
davinci_df = filter(davinci_df, ~_.word.isin(stopwords.words('english')))
davinci_df = filter(davinci_df,~_.word.isin(punctuation_signs))

davinci_df = davinci_df[davinci_df.word.notnull()]
davinci_df = davinci_df.reset_index()
davinci_df = davinci_df.drop(columns=['index'])
davinci_df

,line,chapter,word
0,1,0,da
1,1,0,vinci
2,1,0,code
3,2,0,dan
4,2,0,brown
...,...,...,...
77617,5883,109,2003
77618,5883,109,dan
77619,5883,109,brown
77620,5884,109,isbn


In [30]:
from tidytext import unnest_tokens
import nltk
from nltk.corpus import stopwords
from siuba import *

clouds_df = (unnest_tokens(clouds_df, "word", "content"))
clouds_df = clouds_df.reset_index()
clouds_df = clouds_df.drop(columns=['index'])

clouds_words = count(clouds_df, _.chapter, _.word, sort=True)
clouds_df = filter(clouds_df, ~_.word.isin(stopwords.words('english')))
clouds_df = filter(clouds_df,~_.word.isin(punctuation_signs))

clouds_df = clouds_df[clouds_df.word.notnull()]
clouds_df = clouds_df.reset_index()
clouds_df = clouds_df.drop(columns=['index'])
clouds_df

,line,chapter,word
0,1,0,head
1,3,0,clouds
2,8,0,books
3,8,0,karen
4,8,0,witemeyer
...,...,...,...
51944,253,45,always
51945,253,45,clear
51946,253,45,could
51947,253,45,always


In [31]:
from tidytext import unnest_tokens
import nltk
from nltk.corpus import stopwords
from siuba import *

cosmos_df = (unnest_tokens(cosmos_df, "word", "content"))
cosmos_df = cosmos_df.reset_index()
cosmos_df = cosmos_df.drop(columns=['index'])

cosmos_words = count(cosmos_df, _.chapter, _.word, sort=True)
cosmos_df = filter(cosmos_df, ~_.word.isin(stopwords.words('english')))
cosmos_df = filter(cosmos_df,~_.word.isin(punctuation_signs))

cosmos_df = cosmos_df[cosmos_df.word.notnull()]
cosmos_df = cosmos_df.reset_index()
cosmos_df = cosmos_df.drop(columns=['index'])
cosmos_df

,line,chapter,word
0,0,0,acknowledgments
1,1,0,besides
2,1,0,thanked
3,1,0,introduction
4,1,0,grateful
...,...,...,...
70404,1696,13,2
70405,1696,13,five
70406,1696,13,pythagorean
70407,1696,13,solids
